### Задача:
Проанализировать источники трафика, с которых читатели приходят на статьи блога. Предполагается, что для новых записей в блоге в первые две недели привлекается больше трафика из социальных сетей, чем из других источников. Это позволит понять, позволяют ли публикации в социальных сетях привлечь трафик на старте, когда статья только размещена в блоге и ещё не проиндексирована поисковиками.

In [4]:
token = 'AgAAAAAUU2fzAAZCUgs9ZOhQbUPRhhNcMVv4HA4'
headers = {'Authorization': 'OAuth ' + token}
params = {'metrics': 'ym:s:visits', 
          'dimensions': 'ym:s:date,ym:s:startURLPath,ym:s:lastsignTrafficSource',  
          'date1': '2016-01-01',  
          'date2': '2019-07-01',     
          'ids': 30177909,
          'accuracy':'full',
         'limit':100000}

In [5]:
import json 
import requests
from pprint import pprint
import pandas as pd
from datetime import timedelta

In [6]:
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
print (response.status_code)
metrika_data = response.json()
print(metrika_data.keys())

200
dict_keys(['query', 'data', 'total_rows', 'total_rows_rounded', 'sampled', 'sample_share', 'sample_size', 'sample_space', 'data_lag', 'totals', 'min', 'max'])


In [7]:
print(metrika_data['total_rows'])

15454


In [8]:
pprint(metrika_data['data'][0:5])
metrika_df = pd.DataFrame(metrika_data['data'])
display(metrika_df.head(10))

[{'dimensions': [{'name': '2017-03-08'},
                 {'favicon': 'datalytics.ru',
                  'name': '/all/stanovyas-dzhedaem-logs-api-yandeks-metriki/'},
                 {'icon_id': '8',
                  'icon_type': 'traffic-source',
                  'id': 'social',
                  'name': 'Social network traffic'}],
  'metrics': [154.0]},
 {'dimensions': [{'name': '2018-12-18'},
                 {'favicon': 'datalytics.ru',
                  'name': '/all/kak-ispolzovat-google-bigquery-s-pomoschyu-python/'},
                 {'icon_id': '0',
                  'icon_type': 'traffic-source',
                  'id': 'direct',
                  'name': 'Direct traffic'}],
  'metrics': [127.0]},
 {'dimensions': [{'name': '2018-11-27'},
                 {'favicon': 'datalytics.ru', 'name': '/'},
                 {'icon_id': '0',
                  'icon_type': 'traffic-source',
                  'id': 'direct',
                  'name': 'Direct traffic'}],
  'metrics': [82

,dimensions,metrics
0,"[{'name': '2017-03-08'}, {'name': '/all/stanov...",[154.0]
1,"[{'name': '2018-12-18'}, {'name': '/all/kak-is...",[127.0]
2,"[{'name': '2018-11-27'}, {'name': '/', 'favico...",[82.0]
3,"[{'name': '2019-01-29'}, {'name': '/all/kak-v-...",[62.0]
4,"[{'name': '2018-11-28'}, {'name': '/all/s-cheg...",[56.0]
5,"[{'name': '2017-11-27'}, {'name': '/all/stanov...",[45.0]
6,"[{'name': '2018-08-22'}, {'name': '/all/spisok...",[44.0]
7,"[{'name': '2018-11-29'}, {'name': '/all/s-cheg...",[42.0]
8,"[{'name': '2018-12-18'}, {'name': '/all/kak-is...",[42.0]
9,"[{'name': '2017-03-08'}, {'name': '/all/stanov...",[41.0]


In [6]:
print(metrika_data['query']['metrics'])

['ym:s:visits']


In [7]:
def getMetrikaDataInListOfDicts(metrika_data):
    list_of_dicts = []
    dimensions_list = metrika_data['query']['dimensions']
    metrics_list = metrika_data['query']['metrics']
    for data_item in metrika_data['data']:
        d = {}
        for i,dimension in enumerate(data_item['dimensions']):
            d[dimensions_list[i]] = dimension['name']
        for i,metric in enumerate(data_item['metrics']):
            d[metrics_list[i]] = metric
        list_of_dicts.append(d)
    return list_of_dicts

In [8]:
metrika_list_of_dicts = getMetrikaDataInListOfDicts(metrika_data)
pprint(metrika_list_of_dicts[0:5])

[{'ym:s:date': '2017-03-08',
  'ym:s:lastsignTrafficSource': 'Social network traffic',
  'ym:s:startURLPath': '/all/stanovyas-dzhedaem-logs-api-yandeks-metriki/',
  'ym:s:visits': 154.0},
 {'ym:s:date': '2018-12-18',
  'ym:s:lastsignTrafficSource': 'Direct traffic',
  'ym:s:startURLPath': '/all/kak-ispolzovat-google-bigquery-s-pomoschyu-python/',
  'ym:s:visits': 127.0},
 {'ym:s:date': '2018-11-27',
  'ym:s:lastsignTrafficSource': 'Direct traffic',
  'ym:s:startURLPath': '/',
  'ym:s:visits': 82.0},
 {'ym:s:date': '2019-01-29',
  'ym:s:lastsignTrafficSource': 'Direct traffic',
  'ym:s:startURLPath': '/all/kak-v-pandas-razbit-kolonku-na-neskolko-kolonok/',
  'ym:s:visits': 62.0},
 {'ym:s:date': '2018-11-28',
  'ym:s:lastsignTrafficSource': 'Social network traffic',
  'ym:s:startURLPath': '/all/s-chego-analitiku-nachat-izuchenie-python/',
  'ym:s:visits': 56.0}]


In [9]:
metrika_df = pd.DataFrame(metrika_list_of_dicts)
display(metrika_df.head(10))

,ym:s:date,ym:s:startURLPath,ym:s:lastsignTrafficSource,ym:s:visits
0,2017-03-08,/all/stanovyas-dzhedaem-logs-api-yandeks-metriki/,Social network traffic,154.0
1,2018-12-18,/all/kak-ispolzovat-google-bigquery-s-pomoschy...,Direct traffic,127.0
2,2018-11-27,/,Direct traffic,82.0
3,2019-01-29,/all/kak-v-pandas-razbit-kolonku-na-neskolko-k...,Direct traffic,62.0
4,2018-11-28,/all/s-chego-analitiku-nachat-izuchenie-python/,Social network traffic,56.0
5,2017-11-27,/all/stanovyas-guru-api-yandeks-metriki/,Direct traffic,45.0
6,2018-08-22,/all/spisok-stop-slov-yandeks-direkta/,Search engine traffic,44.0
7,2018-11-29,/all/s-chego-analitiku-nachat-izuchenie-python/,Social network traffic,42.0
8,2018-12-18,/all/kak-ispolzovat-google-bigquery-s-pomoschy...,Social network traffic,42.0
9,2017-03-08,/all/stanovyas-dzhedaem-logs-api-yandeks-metriki/,Direct traffic,41.0


In [10]:
print(metrika_df.shape)

(15454, 4)


In [11]:
print(metrika_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15454 entries, 0 to 15453
Data columns (total 4 columns):
ym:s:date                     15454 non-null object
ym:s:startURLPath             15452 non-null object
ym:s:lastsignTrafficSource    15348 non-null object
ym:s:visits                   15454 non-null float64
dtypes: float64(1), object(3)
memory usage: 483.1+ KB
None


In [12]:
metrika_df['ym:s:date'] = pd.to_datetime(metrika_df['ym:s:date'])
metrika_df['ym:s:visits'] = metrika_df['ym:s:visits'].astype(int)

In [13]:
display(metrika_df[metrika_df['ym:s:lastsignTrafficSource'].isnull()])

,ym:s:date,ym:s:startURLPath,ym:s:lastsignTrafficSource,ym:s:visits
6117,2017-03-13,/,None,3
7213,2016-02-13,/,None,2
7449,2016-07-25,/,None,2
8106,2017-07-20,/all/kak-konvertirovat-json-v-csv/,None,2
8112,2017-07-25,/,None,2
...,...,...,...,...
14601,2019-03-03,/all/kak-ispolzovat-google-bigquery-s-pomoschy...,None,1
14787,2019-03-26,/all/spisok-stop-slov-yandeks-direkta/,None,1
14874,2019-04-07,/all/stanovyas-guru-api-yandeks-metriki/,None,1
15070,2019-05-06,/all/kak-poluchit-spisok-stanciy-moskovskogo-m...,None,1


In [14]:
print(metrika_df['ym:s:visits'].sum())

69994


In [15]:
metrika_df = metrika_df.dropna()

In [16]:
metrika_df.isnull().sum()

ym:s:date                     0
ym:s:startURLPath             0
ym:s:lastsignTrafficSource    0
ym:s:visits                   0
dtype: int64

In [17]:
a =[]
for col in metrika_df.columns:
    a.append(col.replace('ym:s:',''))
metrika_df.columns=a

In [18]:
print(metrika_df['date'].min())
print(metrika_df['date'].max())

2016-01-01 00:00:00
2019-07-01 00:00:00


In [19]:
metrika_df['startURLPath'].value_counts()

/all/stanovyas-guru-api-yandeks-metriki/                                            2630
/all/spisok-stop-slov-yandeks-direkta/                                              2393
/                                                                                   2212
/all/kak-chitat-svoi-soobscheniya-vkontakte-cherez-api/                             1942
/all/kak-konvertirovat-json-v-csv/                                                  1347
                                                                                    ... 
/tags/data-analysis/                                                                   1
/C:/Users/Людмила/Desktop/Список стоп-слов Яндекс.Директа.html                         1
/all/novye-parametry-v-api-yandeks-metriki/comment-1/reply/edit/                       1
/Users/kreatex/Downloads/!чтиво/Становясь падаваном Logs API Яндекс.Метрики.html       1
/drafts/gde-analizirovat-dannye-s-pomoschyu-python/                                    1
Name: startURLPath, L

In [20]:
metrika_df.groupby('lastsignTrafficSource')['visits'].sum()

lastsignTrafficSource
Cached page traffic          45
Direct traffic             9195
Internal traffic            233
Link traffic               1593
Messenger traffic            29
Search engine traffic     56078
Social network traffic     2702
Name: visits, dtype: int32

In [21]:
pages_groupby_df = metrika_df.groupby('startURLPath',as_index=False)['visits'].sum()
pages_w_50_visits = pages_groupby_df[pages_groupby_df['visits'] > 50]['startURLPath'].unique()
print(pages_w_50_visits)
print(len(pages_w_50_visits))

['/' '/all/delaem-sessii-iz-loga-sobytiy-s-pomoschyu-pandas/'
 '/all/golovolomka-pro-randomny-sempl/'
 '/all/kak-chitat-svoi-soobscheniya-vkontakte-cherez-api/'
 '/all/kak-ispolzovat-google-bigquery-s-pomoschyu-python/'
 '/all/kak-konvertirovat-json-v-csv/'
 '/all/kak-poluchit-spisok-stanciy-moskovskogo-metropolitena-po-api/'
 '/all/kak-v-pandas-razbit-kolonku-na-neskolko-kolonok/'
 '/all/novye-parametry-v-api-yandeks-metriki/'
 '/all/probabilistic-programming-and-bayesian-methods-for-hackers/'
 '/all/rabotaem-s-api-google-drive-s-pomoschyu-python/'
 '/all/s-chego-analitiku-nachat-izuchenie-python/'
 '/all/spisok-poleznostey/' '/all/spisok-stop-slov-yandeks-direkta/'
 '/all/stanovyas-dzhedaem-logs-api-yandeks-metriki/'
 '/all/stanovyas-guru-api-yandeks-metriki/' '/search' '/tags/api/']
18


In [22]:
metrika_df = metrika_df[metrika_df['startURLPath'].isin(pages_w_50_visits)]

In [23]:
def category_traffic_sources(traffic_source):
    if traffic_source  == 'Social network traffic':
        category = traffic_source
    else:
        category = 'Other traffic'
    return category

In [24]:
metrika_df['lastsignTrafficCategory'] = metrika_df['lastsignTrafficSource'].apply(category_traffic_sources)
display(metrika_df.head(10))

,date,startURLPath,lastsignTrafficSource,visits,lastsignTrafficCategory
0,2017-03-08,/all/stanovyas-dzhedaem-logs-api-yandeks-metriki/,Social network traffic,154,Social network traffic
1,2018-12-18,/all/kak-ispolzovat-google-bigquery-s-pomoschy...,Direct traffic,127,Other traffic
2,2018-11-27,/,Direct traffic,82,Other traffic
3,2019-01-29,/all/kak-v-pandas-razbit-kolonku-na-neskolko-k...,Direct traffic,62,Other traffic
4,2018-11-28,/all/s-chego-analitiku-nachat-izuchenie-python/,Social network traffic,56,Social network traffic
5,2017-11-27,/all/stanovyas-guru-api-yandeks-metriki/,Direct traffic,45,Other traffic
6,2018-08-22,/all/spisok-stop-slov-yandeks-direkta/,Search engine traffic,44,Other traffic
7,2018-11-29,/all/s-chego-analitiku-nachat-izuchenie-python/,Social network traffic,42,Social network traffic
8,2018-12-18,/all/kak-ispolzovat-google-bigquery-s-pomoschy...,Social network traffic,42,Social network traffic
9,2017-03-08,/all/stanovyas-dzhedaem-logs-api-yandeks-metriki/,Direct traffic,41,Other traffic


In [25]:
min_page_visit_dates = metrika_df.groupby(['startURLPath'],as_index=False)['date'].min()
min_page_visit_dates['date_2_week_ago'] = min_page_visit_dates['date'] + timedelta(days=30)
min_page_visit_dates = min_page_visit_dates.rename(columns={'date':'min_date'})
display(min_page_visit_dates)

,startURLPath,min_date,date_2_week_ago
0,/,2016-01-01,2016-01-31
1,/all/delaem-sessii-iz-loga-sobytiy-s-pomoschyu...,2018-05-05,2018-06-04
2,/all/golovolomka-pro-randomny-sempl/,2019-04-22,2019-05-22
3,/all/kak-chitat-svoi-soobscheniya-vkontakte-ch...,2016-01-01,2016-01-31
4,/all/kak-ispolzovat-google-bigquery-s-pomoschy...,2018-12-18,2019-01-17
5,/all/kak-konvertirovat-json-v-csv/,2016-01-02,2016-02-01
6,/all/kak-poluchit-spisok-stanciy-moskovskogo-m...,2017-01-22,2017-02-21
7,/all/kak-v-pandas-razbit-kolonku-na-neskolko-k...,2019-01-29,2019-02-28
8,/all/novye-parametry-v-api-yandeks-metriki/,2016-06-01,2016-07-01
9,/all/probabilistic-programming-and-bayesian-me...,2016-02-10,2016-03-11


In [26]:
metrika_df = metrika_df.merge(min_page_visit_dates,on='startURLPath',how='left')
display(metrika_df.head(10))

,date,startURLPath,lastsignTrafficSource,visits,lastsignTrafficCategory,min_date,date_2_week_ago
0,2017-03-08,/all/stanovyas-dzhedaem-logs-api-yandeks-metriki/,Social network traffic,154,Social network traffic,2017-03-08,2017-04-07
1,2018-12-18,/all/kak-ispolzovat-google-bigquery-s-pomoschy...,Direct traffic,127,Other traffic,2018-12-18,2019-01-17
2,2018-11-27,/,Direct traffic,82,Other traffic,2016-01-01,2016-01-31
3,2019-01-29,/all/kak-v-pandas-razbit-kolonku-na-neskolko-k...,Direct traffic,62,Other traffic,2019-01-29,2019-02-28
4,2018-11-28,/all/s-chego-analitiku-nachat-izuchenie-python/,Social network traffic,56,Social network traffic,2018-11-28,2018-12-28
5,2017-11-27,/all/stanovyas-guru-api-yandeks-metriki/,Direct traffic,45,Other traffic,2016-01-02,2016-02-01
6,2018-08-22,/all/spisok-stop-slov-yandeks-direkta/,Search engine traffic,44,Other traffic,2016-01-01,2016-01-31
7,2018-11-29,/all/s-chego-analitiku-nachat-izuchenie-python/,Social network traffic,42,Social network traffic,2018-11-28,2018-12-28
8,2018-12-18,/all/kak-ispolzovat-google-bigquery-s-pomoschy...,Social network traffic,42,Social network traffic,2018-12-18,2019-01-17
9,2017-03-08,/all/stanovyas-dzhedaem-logs-api-yandeks-metriki/,Direct traffic,41,Other traffic,2017-03-08,2017-04-07


In [27]:
metrika_df['is_in_2_weeks'] = metrika_df['date'] <= metrika_df['date_2_week_ago']

In [28]:
metrika_df_2 = metrika_df[(metrika_df['is_in_2_weeks'] == True) & (metrika_df['min_date'] != '2016-01-01')]

In [29]:
display(metrika_df_2.head())

,date,startURLPath,lastsignTrafficSource,visits,lastsignTrafficCategory,min_date,date_2_week_ago,is_in_2_weeks
0,2017-03-08,/all/stanovyas-dzhedaem-logs-api-yandeks-metriki/,Social network traffic,154,Social network traffic,2017-03-08,2017-04-07,True
1,2018-12-18,/all/kak-ispolzovat-google-bigquery-s-pomoschy...,Direct traffic,127,Other traffic,2018-12-18,2019-01-17,True
3,2019-01-29,/all/kak-v-pandas-razbit-kolonku-na-neskolko-k...,Direct traffic,62,Other traffic,2019-01-29,2019-02-28,True
4,2018-11-28,/all/s-chego-analitiku-nachat-izuchenie-python/,Social network traffic,56,Social network traffic,2018-11-28,2018-12-28,True
7,2018-11-29,/all/s-chego-analitiku-nachat-izuchenie-python/,Social network traffic,42,Social network traffic,2018-11-28,2018-12-28,True


In [30]:
pivot_df = metrika_df_2.pivot_table(index = 'startURLPath', columns = 'lastsignTrafficCategory', values = 'visits', aggfunc = 'sum')
display(pivot_df)

lastsignTrafficCategory,Other traffic,Social network traffic
startURLPath,,
/all/delaem-sessii-iz-loga-sobytiy-s-pomoschyu-pandas/,59.0,126.0
/all/golovolomka-pro-randomny-sempl/,67.0,28.0
/all/kak-ispolzovat-google-bigquery-s-pomoschyu-python/,424.0,75.0
/all/kak-konvertirovat-json-v-csv/,28.0,NaN
/all/kak-poluchit-spisok-stanciy-moskovskogo-metropolitena-po-api/,47.0,37.0
/all/kak-v-pandas-razbit-kolonku-na-neskolko-kolonok/,219.0,69.0
/all/novye-parametry-v-api-yandeks-metriki/,31.0,95.0
/all/probabilistic-programming-and-bayesian-methods-for-hackers/,6.0,NaN
/all/rabotaem-s-api-google-drive-s-pomoschyu-python/,182.0,50.0


In [31]:
pivot_df['Sum traffic'] = pivot_df['Social network traffic'] + pivot_df['Other traffic']
pivot_df = pivot_df[pivot_df['Sum traffic'] > 1]
pivot_df['is_social_traffic_greater'] = pivot_df['Social network traffic'] > pivot_df['Other traffic']
display(pivot_df)

lastsignTrafficCategory,Other traffic,Social network traffic,Sum traffic,is_social_traffic_greater
startURLPath,,,,
/all/delaem-sessii-iz-loga-sobytiy-s-pomoschyu-pandas/,59.0,126.0,185.0,True
/all/golovolomka-pro-randomny-sempl/,67.0,28.0,95.0,False
/all/kak-ispolzovat-google-bigquery-s-pomoschyu-python/,424.0,75.0,499.0,False
/all/kak-poluchit-spisok-stanciy-moskovskogo-metropolitena-po-api/,47.0,37.0,84.0,False
/all/kak-v-pandas-razbit-kolonku-na-neskolko-kolonok/,219.0,69.0,288.0,False
/all/novye-parametry-v-api-yandeks-metriki/,31.0,95.0,126.0,True
/all/rabotaem-s-api-google-drive-s-pomoschyu-python/,182.0,50.0,232.0,False
/all/s-chego-analitiku-nachat-izuchenie-python/,50.0,304.0,354.0,True
/all/stanovyas-dzhedaem-logs-api-yandeks-metriki/,140.0,290.0,430.0,True


In [32]:
pivot_df['is_social_traffic_greater'].sum()

4

In [33]:
len(pivot_df)

10